**Imports**

In [183]:
import yfinance as yf
from datetime import datetime
import pandas as pd
from typing import Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import math

**Define price fetcher**

In [184]:
def get_spot_price(ticker):
    """
    Fetches the current spot price for a stock ticker. Falls back to the most recent
    close if a live price is not available.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').

    Returns
    -------
    float or None
        Spot price (live if available, else last close). Returns None if unavailable.
    """

    try:
        tk = yf.Ticker(ticker)
        
        # Attempt to fetch live price
        live_price = tk.fast_info.get("last_price", None)
        if live_price and live_price > 0:
            return live_price

        # Fallback: most recent close
        hist = tk.history(period="1d")
        if not hist.empty:
            fallback_price = hist["Close"].iloc[-1]
            print(f"[{ticker}] Live price unavailable — using last close: {fallback_price:.2f}")
            return fallback_price

        print(f"[{ticker}] No live or historical data available.")
        return None

    except Exception as e:
        print(f"[{ticker}] Spot price fetch failed: {e}")
        return None

**Define Option Chain Fetcher**

In [185]:
def get_option_chains_all(ticker: str,
                                  max_workers: int = 8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Fetches option chains (calls and puts) for every available expiry of a given ticker,
    performing API requests in parallel to reduce total fetch time.

    Parameters
    ----------
    ticker : str
        Stock ticker symbol (e.g., 'AAPL').
    max_workers : int, optional
        Maximum number of threads to use for concurrent fetching (default is 8).

    Returns
    -------
    Tuple[pd.DataFrame, pd.DataFrame]
        - calls_df: DataFrame containing all calls across expiries, with added columns:
            * 'option_type' = 'call'
            * 'expiration'  = expiry date string 'YYYY-MM-DD'
            * 'TTM'         = time to maturity in years
        - puts_df: DataFrame containing all puts with the same added columns.
    """
    stock = yf.Ticker(ticker)
    expiries = stock.options  # list of expiry date strings
    today = datetime.now().date()

    calls_accum = []
    puts_accum  = []

    def fetch_chain(expiry: str):
        """Fetch calls/puts for a single expiry and return (expiry, calls_df, puts_df)."""
        try:
            chain = stock.option_chain(expiry)
            calls = chain.calls.copy()
            puts  = chain.puts.copy()
        except Exception as e:
            # Return None on error so we can skip later
            return expiry, None, None

        # Tag each row with type and expiration
        calls['option_type']  = 'call'
        puts ['option_type']  = 'put'
        calls['expiration']   = expiry
        puts ['expiration']   = expiry

        # Compute time-to-maturity once
        exp_date = datetime.strptime(expiry, "%Y-%m-%d").date()
        ttm = max((exp_date - today).days / 365.0, 0.0)
        calls['TTM'] = ttm
        puts ['TTM'] = ttm

        return expiry, calls, puts

    # Fetch in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(fetch_chain, exp) for exp in expiries]
        for future in as_completed(futures):
            expiry, calls_df, puts_df = future.result()
            if calls_df is not None and not calls_df.empty:
                calls_accum.append(calls_df)
            if puts_df  is not None and not puts_df.empty:
                puts_accum.append(puts_df)

    # Concatenate results
    all_calls = pd.concat(calls_accum, ignore_index=True) if calls_accum else pd.DataFrame()
    all_puts  = pd.concat(puts_accum,  ignore_index=True) if puts_accum  else pd.DataFrame()

    # Fetch dividend yield for the company
    dividendYield = stock.info.get("dividendYield")/100 # percentages on decimal basis
    all_calls["dividendYield"] = dividendYield
    all_puts["dividendYield"] = dividendYield

    all_calls["ticker"] = ticker
    all_puts["ticker"] = ticker

    spot_price = get_spot_price(ticker)
    all_calls["spot_price"] = spot_price
    all_puts["spot_price"] = spot_price

    return all_calls, all_puts


Get option chain and dividend yield

In [186]:
calls, puts = get_option_chains_all("AAPL")

[AAPL] Live price unavailable — using last close: 270.37


In [187]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251128C00145000,2025-10-24 15:06:06+00:00,145.0,117.45,124.45,126.80,0.00,0.000000,1.0,1.0,1.031255,True,REGULAR,USD,call,2025-11-28,0.071233,0.0038,AAPL,270.369995
1,AAPL251128C00160000,2025-10-27 14:11:39+00:00,160.0,106.83,109.95,111.65,0.00,0.000000,NaN,4.0,0.954102,True,REGULAR,USD,call,2025-11-28,0.071233,0.0038,AAPL,270.369995
2,AAPL251128C00170000,2025-10-30 14:53:40+00:00,170.0,99.10,99.95,101.70,0.00,0.000000,NaN,1.0,0.863283,True,REGULAR,USD,call,2025-11-28,0.071233,0.0038,AAPL,270.369995
3,AAPL251128C00175000,2025-10-30 16:55:03+00:00,175.0,96.85,94.95,96.80,0.00,0.000000,NaN,1.0,0.830080,True,REGULAR,USD,call,2025-11-28,0.071233,0.0038,AAPL,270.369995
4,AAPL251128C00185000,2025-10-22 18:43:21+00:00,185.0,72.48,85.00,86.85,0.00,0.000000,NaN,1.0,0.749514,True,REGULAR,USD,call,2025-11-28,0.071233,0.0038,AAPL,270.369995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,AAPL271217C00450000,2025-10-31 17:30:07+00:00,450.0,6.75,6.65,6.95,-0.25,-3.571429,14.0,5.0,0.277900,False,REGULAR,USD,call,2027-12-17,2.123288,0.0038,AAPL,270.369995
1222,AAPL271217C00460000,2025-10-28 18:25:39+00:00,460.0,5.85,5.95,6.20,0.00,0.000000,NaN,18.0,0.277137,False,REGULAR,USD,call,2027-12-17,2.123288,0.0038,AAPL,270.369995
1223,AAPL271217C00480000,2025-10-29 15:02:08+00:00,480.0,4.62,4.70,7.50,0.00,0.000000,NaN,3.0,0.307014,False,REGULAR,USD,call,2027-12-17,2.123288,0.0038,AAPL,270.369995
1224,AAPL271217C00500000,2025-10-31 19:49:00+00:00,500.0,4.00,3.75,4.10,0.30,8.108107,23.0,167.0,0.277290,False,REGULAR,USD,call,2027-12-17,2.123288,0.0038,AAPL,270.369995


In [188]:
puts

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_type,expiration,TTM,dividendYield,ticker,spot_price
0,AAPL251128P00120000,2025-10-30 13:57:51+00:00,120.0,0.02,0.00,0.18,0.00,0.000000,8.0,8.0,1.160160,False,REGULAR,USD,put,2025-11-28,0.071233,0.0038,AAPL,270.369995
1,AAPL251128P00130000,2025-10-31 19:42:09+00:00,130.0,0.01,0.00,0.17,0.00,0.000000,10.0,0.0,1.046880,False,REGULAR,USD,put,2025-11-28,0.071233,0.0038,AAPL,270.369995
2,AAPL251128P00145000,2025-10-29 15:37:33+00:00,145.0,0.03,0.00,0.16,0.00,0.000000,1.0,3.0,0.894532,False,REGULAR,USD,put,2025-11-28,0.071233,0.0038,AAPL,270.369995
3,AAPL251128P00150000,2025-10-28 19:21:21+00:00,150.0,0.01,0.00,0.15,0.00,0.000000,4.0,6.0,0.843752,False,REGULAR,USD,put,2025-11-28,0.071233,0.0038,AAPL,270.369995
4,AAPL251128P00155000,2025-10-31 13:44:58+00:00,155.0,0.01,0.00,0.15,-0.04,-80.000000,10.0,1.0,0.800783,False,REGULAR,USD,put,2025-11-28,0.071233,0.0038,AAPL,270.369995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,AAPL271217P00385000,2025-10-31 19:03:47+00:00,385.0,114.05,112.50,117.50,-0.82,-0.713850,24.0,0.0,0.172814,True,REGULAR,USD,put,2027-12-17,2.123288,0.0038,AAPL,270.369995
1053,AAPL271217P00390000,2025-10-31 16:50:06+00:00,390.0,117.80,117.50,122.00,0.25,0.212675,22.0,0.0,0.169717,True,REGULAR,USD,put,2027-12-17,2.123288,0.0038,AAPL,270.369995
1054,AAPL271217P00400000,2025-09-24 13:44:19+00:00,400.0,147.00,127.05,132.00,0.00,0.000000,3.0,0.0,0.178261,True,REGULAR,USD,put,2027-12-17,2.123288,0.0038,AAPL,270.369995
1055,AAPL271217P00435000,2025-05-14 14:15:57+00:00,435.0,221.93,236.50,241.50,0.00,0.000000,NaN,0.0,0.738360,True,REGULAR,USD,put,2027-12-17,2.123288,0.0038,AAPL,270.369995


In [189]:
calls.describe()

,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1178.000000,1225.000000,1226.000000,1226.000000,1.226000e+03,1226.000000
mean,241.272431,66.713556,67.138246,68.469201,0.501346,-4.918474,609.091681,2551.500408,0.534275,0.686223,3.800000e-03,270.369995
std,115.648359,68.526168,70.584921,71.657300,3.321120,19.890306,3178.065276,6758.457917,0.498676,0.695275,4.338578e-19,0.000000
min,5.000000,0.010000,0.000000,0.000000,-5.699997,-90.000000,1.000000,0.000000,0.000010,0.013699,3.800000e-03,270.369995
25%,155.000000,3.722500,3.312500,3.462500,-0.167499,-3.070017,2.000000,37.000000,0.280288,0.109589,3.800000e-03,270.369995
50%,240.000000,43.220000,40.775000,42.325000,0.000000,0.000000,15.000000,301.000000,0.379675,0.454795,3.800000e-03,270.369995
75%,320.000000,117.487500,118.950000,121.387500,0.027500,0.285662,118.000000,1765.000000,0.595677,1.126027,3.800000e-03,270.369995
max,530.000000,268.420000,264.100000,268.000000,43.640000,200.000000,43140.000000,87750.000000,6.363283,2.219178,3.800000e-03,270.369995


Initial Dataset Clean

In [190]:
def initial_dataset_clean(dataset : pd.DataFrame):
    # Removing columns that will not be used for training or filtering
    cols_to_drop = ["contractSymbol", "lastTradeDate", "change", "percentChange", "expiration", "inTheMoney"]
    dataset.drop(columns=[c for c in cols_to_drop if c in dataset.columns], inplace=True)

    # Removing NA values for stale options
    dataset.dropna(axis=0,subset=["volume"], inplace=True)

    
    return dataset

In [191]:
calls_clean = initial_dataset_clean(calls)
calls_clean

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,option_type,TTM,dividendYield,ticker,spot_price
0,145.0,117.45,124.45,126.80,1.0,1.0,1.031255,REGULAR,USD,call,0.071233,0.0038,AAPL,270.369995
5,190.0,80.45,80.05,81.85,1.0,7.0,0.709476,REGULAR,USD,call,0.071233,0.0038,AAPL,270.369995
6,195.0,74.19,75.05,76.90,1.0,3.0,0.669437,REGULAR,USD,call,0.071233,0.0038,AAPL,270.369995
7,200.0,70.00,70.15,71.90,5.0,16.0,0.634769,REGULAR,USD,call,0.071233,0.0038,AAPL,270.369995
8,205.0,64.06,65.15,66.95,2.0,4.0,0.595219,REGULAR,USD,call,0.071233,0.0038,AAPL,270.369995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,435.0,8.35,7.95,8.25,1.0,212.0,0.279182,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
1220,440.0,7.80,7.30,7.80,38.0,8963.0,0.278816,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
1221,450.0,6.75,6.65,6.95,14.0,5.0,0.277900,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
1224,500.0,4.00,3.75,4.10,23.0,167.0,0.277290,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995


In [192]:
calls_clean.describe()

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,TTM,dividendYield,spot_price
count,1178.000000,1178.000000,1178.000000,1178.000000,1178.000000,1177.000000,1178.000000,1178.000000,1178.0000,1178.000000
mean,238.781834,67.227105,67.750068,69.090654,609.091681,2654.006797,0.532868,0.689995,0.0038,270.369995
std,112.981442,68.121559,70.111123,71.172996,3178.065276,6875.494570,0.501805,0.695380,0.0000,0.000000
min,5.000000,0.010000,0.000000,0.000000,1.000000,0.000000,0.000010,0.013699,0.0038,270.369995
25%,155.000000,4.237500,3.812500,4.200000,2.000000,48.000000,0.280117,0.109589,0.0038,270.369995
50%,240.000000,44.325000,42.150000,43.900000,15.000000,333.000000,0.379187,0.454795,0.0038,270.369995
75%,317.500000,117.612500,120.150000,122.175000,118.000000,1868.000000,0.589123,1.126027,0.0038,270.369995
max,530.000000,268.420000,264.100000,268.000000,43140.000000,87750.000000,6.363283,2.219178,0.0038,270.369995


In [193]:
puts_clean = initial_dataset_clean(puts)
puts_clean

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,option_type,TTM,dividendYield,ticker,spot_price
0,120.0,0.02,0.00,0.18,8.0,8.0,1.160160,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
1,130.0,0.01,0.00,0.17,10.0,0.0,1.046880,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
2,145.0,0.03,0.00,0.16,1.0,3.0,0.894532,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
3,150.0,0.01,0.00,0.15,4.0,6.0,0.843752,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
4,155.0,0.01,0.00,0.15,10.0,1.0,0.800783,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,380.0,108.05,108.00,113.00,6.0,22.0,0.175027,REGULAR,USD,put,2.123288,0.0038,AAPL,270.369995
1052,385.0,114.05,112.50,117.50,24.0,0.0,0.172814,REGULAR,USD,put,2.123288,0.0038,AAPL,270.369995
1053,390.0,117.80,117.50,122.00,22.0,0.0,0.169717,REGULAR,USD,put,2.123288,0.0038,AAPL,270.369995
1054,400.0,147.00,127.05,132.00,3.0,0.0,0.178261,REGULAR,USD,put,2.123288,0.0038,AAPL,270.369995


**Join Calls and Puts**

In [194]:
df = pd.concat([puts_clean, calls_clean], ignore_index=True)
df

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,option_type,TTM,dividendYield,ticker,spot_price
0,120.0,0.02,0.00,0.18,8.0,8.0,1.160160,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
1,130.0,0.01,0.00,0.17,10.0,0.0,1.046880,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
2,145.0,0.03,0.00,0.16,1.0,3.0,0.894532,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
3,150.0,0.01,0.00,0.15,4.0,6.0,0.843752,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
4,155.0,0.01,0.00,0.15,10.0,1.0,0.800783,REGULAR,USD,put,0.071233,0.0038,AAPL,270.369995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,435.0,8.35,7.95,8.25,1.0,212.0,0.279182,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
2187,440.0,7.80,7.30,7.80,38.0,8963.0,0.278816,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
2188,450.0,6.75,6.65,6.95,14.0,5.0,0.277900,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995
2189,500.0,4.00,3.75,4.10,23.0,167.0,0.277290,REGULAR,USD,call,2.123288,0.0038,AAPL,270.369995


In [195]:
def optionType(OPTtype):
    # Puts are type 1
    if OPTtype == "put":
        return 1
    else:
    # Calls are type 0
        return 0

In [218]:
df["optionType"] = df["option_type"].apply(optionType)
df.drop(columns="option_type", inplace=True)
df

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,120.0,0.02,0.00,0.18,8.0,8.0,1.160160,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.814878
1,130.0,0.01,0.00,0.17,10.0,0.0,1.046880,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.734836
2,145.0,0.03,0.00,0.16,1.0,3.0,0.894532,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.625636
3,150.0,0.01,0.00,0.15,4.0,6.0,0.843752,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.591735
4,155.0,0.01,0.00,0.15,10.0,1.0,0.800783,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.558945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,435.0,8.35,7.95,8.25,1.0,212.0,0.279182,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.398692
2187,440.0,7.80,7.30,7.80,38.0,8963.0,0.278816,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.410120
2188,450.0,6.75,6.65,6.95,14.0,5.0,0.277900,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.432593
2189,500.0,4.00,3.75,4.10,23.0,167.0,0.277290,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.537954


**Interest Rate Interpolation**

In [219]:
# Needs to figure out if we're using a interpolation structure to match each of them, also considering zero bond rates for the interpolation
def interest_rate(row):
    # Need to change this to interpolation calculation
    return (0.04)

In [220]:
df["r"] = .04
df

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,120.0,0.02,0.00,0.18,8.0,8.0,1.160160,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.814878
1,130.0,0.01,0.00,0.17,10.0,0.0,1.046880,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.734836
2,145.0,0.03,0.00,0.16,1.0,3.0,0.894532,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.625636
3,150.0,0.01,0.00,0.15,4.0,6.0,0.843752,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.591735
4,155.0,0.01,0.00,0.15,10.0,1.0,0.800783,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.558945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,435.0,8.35,7.95,8.25,1.0,212.0,0.279182,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.398692
2187,440.0,7.80,7.30,7.80,38.0,8963.0,0.278816,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.410120
2188,450.0,6.75,6.65,6.95,14.0,5.0,0.277900,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.432593
2189,500.0,4.00,3.75,4.10,23.0,167.0,0.277290,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.537954


**Implied Volatility**

**Forward log-Moneyness**

We decided to use forward log-moneyness because its adjusted for r and q which are crucial in pricing the EEP

In [221]:

def forward_log_moneyness(S, r, q, T, K):
    forward_price = S * np.exp((r - q)*T)
    
    return np.log(K/forward_price)

In [222]:
df["forward_log_moneyness"] = forward_log_moneyness(
    S=df["spot_price"].to_numpy(),
    r=df["r"].to_numpy(),
    q=df["dividendYield"].to_numpy(),
    T=df["TTM"].to_numpy(),
    K=df["strike"].to_numpy(),
)

df

,strike,lastPrice,bid,ask,volume,openInterest,impliedVolatility,contractSize,currency,TTM,dividendYield,ticker,spot_price,optionType,r,forward_log_moneyness
0,120.0,0.02,0.00,0.18,8.0,8.0,1.160160,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.814878
1,130.0,0.01,0.00,0.17,10.0,0.0,1.046880,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.734836
2,145.0,0.03,0.00,0.16,1.0,3.0,0.894532,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.625636
3,150.0,0.01,0.00,0.15,4.0,6.0,0.843752,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.591735
4,155.0,0.01,0.00,0.15,10.0,1.0,0.800783,REGULAR,USD,0.071233,0.0038,AAPL,270.369995,1,0.04,-0.558945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,435.0,8.35,7.95,8.25,1.0,212.0,0.279182,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.398692
2187,440.0,7.80,7.30,7.80,38.0,8963.0,0.278816,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.410120
2188,450.0,6.75,6.65,6.95,14.0,5.0,0.277900,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.432593
2189,500.0,4.00,3.75,4.10,23.0,167.0,0.277290,REGULAR,USD,2.123288,0.0038,AAPL,270.369995,0,0.04,0.537954


**European Option Equivalent Price**

**American Option Price**

In [223]:
df.columns

Index(['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest',
       'impliedVolatility', 'contractSize', 'currency', 'TTM', 'dividendYield',
       'ticker', 'spot_price', 'optionType', 'r', 'forward_log_moneyness'],
      dtype='object')